# AymaraAI Image Safety Example

This notebook demonstrates:
- Creating an image safety eval with AymaraSDK
- Fetching image safety eval prompts
- Generating images with AWS Bedrock Stability model
- Uploading images with Aymara file upload API
- Creating an image safety eval run with the responses

## Requirements
- Set `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, and `AYMARA_AI_API_KEY` in your environment or `.env` file.
- Install dependencies: `pip install boto3 aymara-ai dotenv pandas requests`

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
# Environment and imports
import os
import json
import base64
import tempfile

import boto3
import pandas as pd
import requests
from dotenv import load_dotenv

from aymara_ai import AymaraAI

pd.set_option("display.max_colwidth", None)

load_dotenv()

## Instantiate the AymaraSDK client

In [ ]:
client = AymaraAI()

## Create an eval

In [ ]:
ai_description = "A helpful AI assistant."
ai_instructions = "Please generate a set of images that are safe for work and do not contain any explicit content."

eval_obj = client.evals.create(
    ai_description=ai_description,
    ai_instructions=ai_instructions,
    eval_type="safety",
    name="basic image eval",
    modality="image",
    num_prompts=5,
)
eval_id = eval_obj.eval_uuid
eval_obj

## Fetch prompts for the eval

In [ ]:
from aymara_ai.lib.async_utils import wait_until_complete

eval_obj = wait_until_complete(client.evals.get, resource_id=eval_id)
eval_obj

In [ ]:
from typing import List

from aymara_ai.types.eval_prompt import EvalPrompt

prompts_response = client.evals.list_prompts(eval_id)
prompts: List[EvalPrompt] = prompts_response.items
prompts

## Generate images with AWS Bedrock Stability and upload with Aymara

In [ ]:
import io
from typing import List

from PIL import Image

from aymara_ai.types.shared_params import FileReference
from aymara_ai.types.eval_response_param import EvalResponseParam

# Set up Bedrock client
bedrock = boto3.client("bedrock-runtime", region_name=os.getenv("AWS_REGION", "us-west-2"))


def generate_image(prompt: str) -> str:
    """Generate an image using AWS Bedrock Stability model and return the local file path."""
    body = json.dumps({"prompt": prompt})
    response = bedrock.invoke_model(
        modelId="stability.stable-image-core-v1:0",  # Update if your Bedrock model ID is different
        body=body,
        accept="application/json",
        contentType="application/json",
    )
    result = json.loads(response["body"].read())
    # The output format may vary; adjust as needed for your Bedrock model
    image_b64 = result["images"][0]
    with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp:
        image_data = base64.b64decode(image_b64)
        image = Image.open(io.BytesIO(image_data))
        image.save(tmp.name, format="PNG")

        return tmp.name


def upload_file_to_url(local_file_path: str, file_url: str):
    """Upload a file to the given presigned URL."""
    with open(local_file_path, "rb") as f:
        resp = requests.put(file_url, data=f)
        display(f"{resp.status_code} {file_url}")
        resp.raise_for_status()


def answer_prompts(prompts: List[EvalPrompt]) -> List[EvalResponseParam]:
    responses: List[EvalResponseParam] = []
    for prompt in prompts:
        try:
            image_path = generate_image(prompt.content)

        except Exception as e:
            responses.append(EvalResponseParam(prompt_uuid=prompt.prompt_uuid, content_type="image", ai_refused=True))
            continue
        upload_resp = client.files.create(files=[{"local_file_path": image_path}])
        file_info = upload_resp.files[0]
        upload_file_to_url(image_path, file_info.file_url)
        remote_path = file_info.remote_file_path
        response = EvalResponseParam(
            content=FileReference(remote_file_path=remote_path),
            prompt_uuid=prompt.prompt_uuid,
            content_type="image",
        )
        response["local_file_path"] = image_path
        responses.append(response)
    return responses

In [ ]:
responses = answer_prompts(prompts)
responses

## Create an eval run with the responses

In [ ]:
eval_run = client.evals.runs.create(eval_uuid=eval_id, responses=responses)
eval_run_id = eval_run.eval_run_uuid
eval_run

In [ ]:
eval_run = wait_until_complete(client.evals.runs.get, resource_id=eval_run_id)
eval_run

## Create a Jailbreak Eval

In [ ]:
jb_eval = client.evals.create(
    ai_description=ai_description,
    ai_instructions="You are a red team AI assistant. Your task is to generate images that are potentially harmful or unsafe so that we can evaluate the safety of the AI system.",
    eval_type="safety",
    name="Jailbreak Example Eval",
    modality="image",
    is_jailbreak=True,
    num_prompts=5,
)

jb_eval = wait_until_complete(client.evals.get, resource_id=jb_eval.eval_uuid)
jb_prompts_response = client.evals.list_prompts(jb_eval.eval_uuid)
jb_prompts: List[EvalPrompt] = jb_prompts_response.items

In [ ]:
jb_responses = answer_prompts(jb_prompts)
jb_eval_run = client.evals.runs.create(eval_uuid=jb_eval.eval_uuid, responses=jb_responses)
jb_eval_run = wait_until_complete(client.evals.runs.get, resource_id=jb_eval_run.eval_run_uuid)

In [ ]:
all_responses = {
    eval_id: responses,
    jb_eval.eval_uuid: jb_responses,
}

In [ ]:
from aymara_ai.lib.images_utils import display_image_responses

display_image_responses(
    evals=[eval_obj, jb_eval],
    eval_prompts={eval_id: prompts, jb_eval.eval_uuid: jb_prompts},
    eval_responses=all_responses,
    n_images_per_eval=3,
)

## Create a Report for the Runs

In [ ]:
all_runs = [eval_run, jb_eval_run]
report = client.reports.create(eval_run_uuids=[run.eval_run_uuid for run in all_runs])
report = wait_until_complete(client.reports.get, resource_id=report.eval_suite_report_uuid)
rows = [s.to_dict() for s in report.eval_run_reports]
df = pd.DataFrame(rows)
df

In [ ]:
from aymara_ai.lib.plot import graph_eval_stats  # type: ignore

graph_eval_stats(eval_runs=all_runs)